In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

# ✅ Load Training Data
try:
    store_sales = pd.read_csv(
        "/kaggle/input/store-sales-time-series-forecasting-2/train.csv",
        usecols=['store_nbr', 'family', 'date', 'sales'],
        dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
        parse_dates=['date'],
    )
    store_sales['date'] = store_sales.date.dt.to_period('D')
    store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()
    print("📌 Training Data Sample:\n", store_sales.head())
except FileNotFoundError as e:
    print(f"Error loading train.csv: {e}")

# Prepare target (y) - unstack to have store-family combinations as columns
y = store_sales['sales'].unstack(['store_nbr', 'family']).loc["2017"]

# Split into training (Jan 1 - Jul 31, 2017) and validation (Aug 1 - Aug 15, 2017)
y_train = y.loc['2017-01-01':'2017-07-31']
y_val = y.loc['2017-08-01':'2017-08-15']

# Calculate average sales and multiple lags
avg_sales = store_sales['sales'].groupby('date').mean().loc["2016":"2017"]
lag_sales_1 = avg_sales.shift(1).loc["2017"].rename('lag_sales_1')
lag_sales_2 = avg_sales.shift(2).loc["2017"].rename('lag_sales_2')
lag_sales_7 = avg_sales.shift(7).loc["2017"].rename('lag_sales_7')

# ✅ Load Holiday Data
try:
    holidays_events = pd.read_csv(
        "/kaggle/input/store-sales-time-series-forecasting-2/holidays_events.csv",
        dtype={'type': 'category', 'locale': 'category', 'locale_name': 'category', 'description': 'category', 'transferred': 'bool'},
        parse_dates=['date'],
    )
    holidays_events['date'] = holidays_events.date.dt.to_period('D')
    print("Loaded holidays_events.csv successfully")
except FileNotFoundError as e:
    print(f"Error loading holidays_events.csv: {e}")

# Filter national and regional holidays
holidays = holidays_events.query("locale in ['National', 'Regional']").loc['2017':'2017-08-31', ['description']]
holidays = holidays.assign(description=lambda x: x.description.cat.remove_unused_categories())

# ✅ Load Oil Data
try:
    oil = pd.read_csv(
        "/kaggle/input/store-sales-time-series-forecasting-2/oil.csv",
        parse_dates=['date'],
    ).set_index('date').to_period('D')
    oil['dcoilwtico'] = oil['dcoilwtico'].ffill().bfill()
    print("Loaded oil.csv successfully")
except FileNotFoundError as e:
    print(f"Error loading oil.csv: {e}")

# ✅ Prepare Training Features
fourier = CalendarFourier(freq='ME', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X = dp.in_sample()

# Split features into train and validation
X_train = X.loc['2017-01-01':'2017-07-31']
X_val = X.loc['2017-08-01':'2017-08-15']

# Add New Year indicator
X['NewYear'] = (X.index.dayofyear == 1)

# Add holiday features
X_holidays = pd.get_dummies(holidays, columns=['description'], dtype=float)
X = X.join(X_holidays, on='date').fillna(0.0)

# Add oil price feature
X = X.join(oil['dcoilwtico'], on='date').ffill().bfill()

# Add multiple lagged sales features
X = X.join(lag_sales_1, on='date').join(lag_sales_2, on='date').join(lag_sales_7, on='date')
X[['lag_sales_1', 'lag_sales_2', 'lag_sales_7']] = X[['lag_sales_1', 'lag_sales_2', 'lag_sales_7']].fillna(avg_sales.mean())

# Verify no NaNs remain
if X.isnull().any().any():
    print("Warning: NaNs found in X after preprocessing:")
    print(X.isnull().sum())
    raise ValueError("NaNs still present in training features")

# Split X again after adding features
X_train = X.loc['2017-01-01':'2017-07-31']
X_val = X.loc['2017-08-01':'2017-08-15']

print("📌 Final Training Features (X_train):\n", X_train.head())

# ✅ Train Model
model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

# ✅ Evaluate on Validation Set
y_val_pred = pd.DataFrame(model.predict(X_val), index=X_val.index, columns=y_val.columns)

# Compute evaluation metrics
# RMSLE (clip negative predictions to 0 as log can't handle negatives)
y_val_true = y_val.stack(['store_nbr', 'family']).values
y_val_pred_flat = y_val_pred.stack(['store_nbr', 'family']).values
y_val_pred_flat = np.clip(y_val_pred_flat, 0, None)  # Avoid negative predictions
rmsle = np.sqrt(mean_squared_log_error(y_val_true, y_val_pred_flat))

# MAE
mae = mean_absolute_error(y_val_true, y_val_pred_flat)

# R² Score
r2 = r2_score(y_val_true, y_val_pred_flat)

print("\n📌 Validation Metrics:")
print(f"RMSLE: {rmsle:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R² Score: {r2:.4f}")

# Check in-sample predictions (on full training data for submission)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)
print("📌 Sample Training Predictions (y_pred):\n", y_pred.head())

# ✅ Load Test Data While Preserving 'id'
try:
    df_test = pd.read_csv(
        "/kaggle/input/store-sales-time-series-forecasting-2/test.csv",
        dtype={'store_nbr': 'category', 'family': 'category'},
        parse_dates=['date'],
    )
    df_test_id = df_test[['id', 'store_nbr', 'family', 'date']].copy()
    df_test['date'] = df_test.date.dt.to_period('D')
    df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()
    print("Loaded test.csv successfully")
except FileNotFoundError as e:
    print(f"Error loading test.csv: {e}")

# ✅ Generate Test Features
test_dates = df_test.index.get_level_values('date').unique()
X_test = dp.out_of_sample(steps=len(test_dates))
X_test.index = test_dates
X_test.index.name = 'date'

# Add New Year indicator
X_test['NewYear'] = (X_test.index.dayofyear == 1)

# Add holiday features
X_test = X_test.join(X_holidays, on='date').fillna(0.0)

# Add oil price feature
X_test = X_test.join(oil['dcoilwtico'], on='date').ffill().bfill()

# Add lagged sales features for test period (using last available training values)
last_lag_1 = avg_sales.loc['2017-08-15']
last_lag_2 = avg_sales.loc['2017-08-14']
last_lag_7 = avg_sales.loc['2017-08-09']
X_test['lag_sales_1'] = last_lag_1
X_test['lag_sales_2'] = last_lag_2
X_test['lag_sales_7'] = last_lag_7

# Ensure feature consistency
missing_cols = set(X.columns) - set(X_test.columns)
for col in missing_cols:
    X_test[col] = 0.0
X_test = X_test[X.columns]

# Verify no NaNs in test features
if X_test.isnull().any().any():
    print("Warning: NaNs found in X_test after preprocessing:")
    print(X_test.isnull().sum())
    raise ValueError("NaNs still present in test features")

print("📌 Final Test Features (X_test):\n", X_test.head())

# ✅ Make Predictions
y_submit = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
y_submit = y_submit.stack(['store_nbr', 'family'], future_stack=True).reset_index(name='sales')

# ✅ Fix ID Loss Issue
df_test_id['date'] = df_test_id['date'].astype(str)
y_submit['date'] = y_submit['date'].astype(str)
y_submit = df_test_id.merge(y_submit, on=['store_nbr', 'family', 'date'], how='left')
y_submit['sales'] = y_submit['sales'].fillna(0)
y_submit = y_submit[['id', 'sales']]

# ✅ Verify and Save Submission
print("\n📌 Sample of submission.csv with All Test IDs Preserved:")
print(y_submit.head(10))
y_submit.to_csv('/kaggle/working/submission.csv', index=False)
print("📌 Submission file generated successfully! 📌")

📌 Training Data Sample:
                                  sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0
Loaded holidays_events.csv successfully
Loaded oil.csv successfully
📌 Final Training Features (X_train):
             const  trend  s(2,7)  s(3,7)  s(4,7)  s(5,7)  s(6,7)  s(7,7)  \
date                                                                       
2017-01-01    1.0    1.0     0.0     0.0     0.0     0.0     0.0     0.0   
2017-01-02    1.0    2.0     1.0     0.0     0.0     0.0     0.0     0.0   
2017-01-03    1.0    3.0     0.0     1.0     0.0     0.0     0.0     0.0   
2017-01-04    1.0    4.0     0.0     0.0     1.0     0.0     0.0     0.0   
2017-01-05    1.0    5.0     0.0     0.0     0.0     1.0     0.0     0.0   

            sin(1,freq=ME)  cos(1,freq=ME)  ...  cos(2,freq=ME)

<ipython-input-1-29fb0710df78>:112: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  y_val_true = y_val.stack(['store_nbr', 'family']).values
<ipython-input-1-29fb0710df78>:113: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  y_val_pred_flat = y_val_pred.stack(['store_nbr', 'family']).values



📌 Validation Metrics:
RMSLE: 0.5549
MAE: 84.4390
R² Score: 0.9472
📌 Sample Training Predictions (y_pred):
 store_nbr           1                                                          \
family     AUTOMOTIVE BABY CARE    BEAUTY    BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                            
2017-01-01   0.986276       0.0  1.499982   640.197814  0.270524   110.836785   
2017-01-02   7.739112       0.0  2.254279  1693.741211  0.445458   315.805015   
2017-01-03   3.898980       0.0  3.028823  2743.087406  1.285447   457.411317   
2017-01-04   3.245901       0.0  4.519788  2759.927983  0.985035   503.967718   
2017-01-05   3.623657       0.0  3.167645  2278.896405  0.702343   427.633712   

store_nbr                                                    ...         9  \
family     CELEBRATION     CLEANING       DAIRY        DELI  ... MAGAZINES   
date                                                         ...             
2017-01-0